In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

import getpass
pd.options.mode.chained_assignment = None

In [2]:
# Ruta para pcs
user = getpass.getuser() # Capturo el usuario local / Ejemplo "Llan_", este es el nombre de mi usuario que inicie sesion en la pc

if   user== "USUARIO": # laptop @hugoferquiroz
     work = Path(r"D:\\Trabajo\Brecha-PRITE")
     data = Path(r"D:\\OneDrive\\Bases de datos\\Minedu compartido")
elif user== "hugo_": # laptop @hugoferquiroz
     work = Path(r"C:\\Users\\hugo_\\OneDrive - Ministerio de Educación\\MINEDU_2022\\RACIO")
     data = Path(r"C:\\OneDrive\\Bases de datos\\Minedu compartido")

# Construcción de la base de datos

## Matricula

In [4]:
# Importamos las bases de datos
df_mat_20 = pd.read_excel(data/r'matricula prite\\AC - Reporte de matricula por edad en los PRITE 2020-2022 - SIAGIE.xlsx',sheet_name='2020',skiprows=3)
df_mat_21 = pd.read_excel(data/r'matricula prite\\AC - Reporte de matricula por edad en los PRITE 2020-2022 - SIAGIE.xlsx',sheet_name='2021',skiprows=3)
df_mat_22 = pd.read_excel(data/r'matricula prite\\AC - Reporte de matricula por edad en los PRITE 2020-2022 - SIAGIE.xlsx',sheet_name='2022',skiprows=3)

In [5]:
def clean_matricula(df, año):
    # Cambiar los nombres
    df.rename(columns={0:'Menor a 12 meses',
                        1:'Entre 12 meses a 1 año',
                        2:'Entre 1 año a 2 años',
                        3:'Entre 2 años a 3 años',
                        4:'Entre 4 años a 5 años',
                        5:'Entre 5 años a 6 años',
                        'TOTAL':'Matricula reportada'}, inplace=True) 
    # Eliminar columnas innecesarias    
    if 'ERROR EN REGISTRO' in  df.columns.to_list():
        df.drop(columns=['ERROR EN REGISTRO'], inplace=True)
    # Verificar la longitud del codigo modular
    df['COD_MOD'] = df['COD_MOD'].apply(lambda x: x.zfill(7))
    # Eliminar la ultima fila = fila del total
    df.drop(df[df['COD_MOD'].str.find('TOTAL')!=-1].index,inplace=True)
    # Completar los missing de la matricula con 0
    df.fillna(0, inplace=True)
    # Agregar el año de la base
    df['Año'] = año
    return df

In [6]:
# Limpio las bases
df_mat_2020 = clean_matricula(df_mat_20,2020)
df_mat_2021 = clean_matricula(df_mat_21,2021)
df_mat_2022 = clean_matricula(df_mat_22,2022)

In [7]:
# Consolido las bases
matricula = pd.concat([df_mat_2020,df_mat_2021])
matricula = pd.concat([matricula,df_mat_2022])

In [8]:
# Doy formato a las columnas
    ## Genero las columnas en el orden que quiero
columnas = matricula.columns.to_list()
columnas_matricula = [columnas[0]] + [columnas[-3]] + columnas[1:5] + columnas[-2:] + [columnas[-4]]
columnas_matricula
    ## Cambio el orden del DataFrame
matricula = matricula[columnas_matricula]
    ## Imputo 0 a los missing\
matricula.fillna(0,inplace=True)

In [9]:
matricula.sample(5)

,COD_MOD,Año,Menor a 12 meses,Entre 12 meses a 1 año,Entre 1 año a 2 años,Entre 2 años a 3 años,Entre 4 años a 5 años,Entre 5 años a 6 años,Matricula reportada
74,1731686,2020,6.0,5.0,5.0,0.0,0.0,0.0,16
62,1627397,2022,10.0,9.0,16.0,0.0,0.0,0.0,35
42,1410414,2020,3.0,7.0,6.0,0.0,0.0,0.0,16
78,1731777,2021,4.0,6.0,11.0,0.0,0.0,0.0,21
36,1386473,2022,3.0,2.0,5.0,0.0,0.0,0.0,10


Evolución de la matricula

In [69]:
matricula_año = matricula.groupby('Año', as_index=False)['Matricula reportada'].agg('sum')

In [71]:
matricula_año.to_excel(work/r'tablas\\Evolución de matricula.xlsx')

## Personal

NEXUS al 4 de setiembre del 2022

In [10]:
# Importo la base de datos
nexus = pd.read_stata(data/r'Nexus\\2022\\nexus_35sira.dta')

c:\Users\USUARIO\anaconda3\lib\site-packages\pandas\io\stata.py:1514: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


In [11]:
# Selecciono los ssee identificados como PRITE
ssee_prite = matricula[['COD_MOD']]
ssee_prite.drop_duplicates(inplace=True) 
personal = pd.merge(left= ssee_prite, right=nexus, left_on='COD_MOD', right_on='codmod', how='left', validate='1:m',indicator=True)

In [12]:
# Funcion para identificar los duplicados por orden de prioridad
def plaza_unica(dataframe,codigo_modular='codmod'):
    print('--------------------------------------------------------------')
    print('Transformación de variables')
    dataframe['codmod'] = dataframe['codmod'].apply(str)
    dataframe.rename(columns={'codmod':'cod_mod'},inplace=True)
    dataframe['cod_mod'] = dataframe['cod_mod'].apply(lambda x: x.zfill(7))
    print('1. Verificar cantidad de digitos código modular')
    
    dataframe = dataframe[(dataframe.sitlab != "B") & (dataframe.sitlab != "P")]
    print('2. Eliminar plazas bloqueadas y previstas')
    
    dataframe['prior_tipo'] = 1
    dataframe.loc[dataframe['tiporegistro']== 'EVENTUAL', 'prior_tipo'] = 2
    dataframe.loc[dataframe['tiporegistro']== 'PROYECTO', 'prior_tipo'] = 3
    dataframe.loc[dataframe['tiporegistro']== 'CUADRO DE HORAS', 'prior_tipo'] = 4
    dataframe.loc[dataframe['tiporegistro'].str.find('REEMPLAZO')!=-1, 'prior_tipo'] = 5
    print('3. Priorizacion del tipo de plaza')

    dataframe['prior_sitlab'] = 1
    dataframe.loc[(dataframe['sitlab']== 'F') | (dataframe['sitlab']== 'D') 
                 | (dataframe['sitlab']== 'E') | (dataframe['sitlab']== 'T'), 'prior_sitlab'] = 2
    dataframe.loc[(dataframe['sitlab']== 'C') | (dataframe['sitlab']== 'V'), 'prior_sitlab'] = 3
    print('4. Priorizacion de la situación laboral')

    dataframe['soplaza'] = 0
    dataframe.loc[((dataframe['estplaza'].str.find("SG")==-1) 
                  & (dataframe['estplaza'].str.find("CG")==-1) 
                  & (dataframe['estplaza'].str.find("ABAND")==-1)), 'soplaza'] = 1
    print('5. Identificación plaza con goce, sin goce')
    
    dataframe['sestpla'] = 0
    dataframe.loc[dataframe['estplaza']== 'ACTIV', 'sestpla'] = 1
    print('6. Identificación estado de la plaza')
    
    dataframe.loc[dataframe['jornlab']=='', 'jornlab'] = 99
    dataframe = dataframe.sort_values(by='jornlab', ascending=False)
    dataframe['sjornlb'] = dataframe['jornlab'].rank(method='dense', ascending=False)
    print('7. Ordenamiento descendente de la jornada laboral')

    print('--------------------------------------------------------------')
    dataframe = dataframe.sort_values(['descreg', 'nombreooii', 'cod_mod', 'codplaza', 'prior_tipo', 
                          'prior_sitlab', 'soplaza', 'sestpla', 'sjornlb'])
    dataframe['tipo_fin'] = dataframe.groupby(['cod_mod', 'codplaza']).cumcount()
    dataframe['tipo_fin'] = dataframe['tipo_fin'] + 1
    print('Identificación plaza única')
    print(dataframe.groupby(['tipo_fin'])['tipo_fin'].count())
    print('Plaza original tienen la etiqueta igual a 1')
    return dataframe


In [13]:
# Identifico los duplicados
personal = plaza_unica(personal)

--------------------------------------------------------------
Transformación de variables
1. Verificar cantidad de digitos código modular
2. Eliminar plazas bloqueadas y previstas
3. Priorizacion del tipo de plaza
4. Priorizacion de la situación laboral
5. Identificación plaza con goce, sin goce
6. Identificación estado de la plaza
7. Ordenamiento descendente de la jornada laboral
--------------------------------------------------------------
Identificación plaza única
tipo_fin
1    519
2     28
3      1
4      1
Name: tipo_fin, dtype: int64
Plaza original tienen la etiqueta igual a 1


In [14]:
# Elimino los duplicados
personal = personal.loc[(personal['tipo_fin']==1) | (personal['_merge']=='left_only')]

Personal por regimen laboral

In [15]:
pd.crosstab(personal['descargo'],personal['desley'],margins=True)

desley,D.L. 1153,D.L. 276,DL 1057,LEY 29944,LEY 30328,LEY 30493,All
descargo,,,,,,,
AUXILIAR DE EDUCACION,0,0,0,0,0,8,8
COORDINACION PEDAGOGICO,0,0,0,1,0,0,1
COORDINADOR ACADEMICO,0,0,0,5,0,0,5
DIRECTOR I.E.,0,0,0,33,0,0,33
DOCENTE COORDINADOR,0,0,0,0,1,0,1
FISIOTERAPEUTA,0,1,0,0,0,0,1
OFICINISTA,0,2,0,0,0,0,2
PROFESIONAL DE TERAPIA FÍSICA PARA LOS PROGRAMAS DE INTERVENCIÓN TEMPRANA,0,0,123,0,0,0,123
PROFESOR,0,0,0,121,146,0,267


In [16]:
# Exporto la tabla
(pd.crosstab(personal['descargo'],personal['desley'],margins=True)).to_excel(work/r'tablas\\personal por regimen laboral.xlsx')

Personal por tipo de trabajador

In [17]:
pd.crosstab(personal['descargo'],personal['desctipotrab'],margins=True)

desctipotrab,ADMINISTRATIVO,CAS,DOCENTE,PROFESIONAL DE LA SALUD,All
descargo,,,,,
AUXILIAR DE EDUCACION,0,0,8,0,8
COORDINACION PEDAGOGICO,0,0,1,0,1
COORDINADOR ACADEMICO,0,0,5,0,5
DIRECTOR I.E.,0,0,33,0,33
DOCENTE COORDINADOR,0,0,1,0,1
FISIOTERAPEUTA,1,0,0,0,1
OFICINISTA,2,0,0,0,2
PROFESIONAL DE TERAPIA FÍSICA PARA LOS PROGRAMAS DE INTERVENCIÓN TEMPRANA,0,123,0,0,123
PROFESOR,0,0,267,0,267


In [18]:
(pd.crosstab(personal['descargo'],personal['desctipotrab'],margins=True)).to_excel(work/r'tablas\\personal por tipo de trabajador.xlsx')

In [19]:
# Homogenizo los perfiles
    ## Docentes
personal['Profesor'] = 0
personal.loc[(personal['descargo']=='PROFESOR') | 
             (personal['descargo']=='PROFESOR (FUNCIONES DE DIRECTOR)') | 
             (personal['descargo']=='PROFESOR COORDINADOR') |
             (personal['descargo']=='COORDINACION PEDAGOGICO') |
             (personal['descargo']=='AUXILIAR DE EDUCACION') | 
             (personal['descargo']=='DOCENTE COORDINADOR') |        
             (personal['descargo']=='COORDINADOR ACADEMICO') ,'Profesor'] =1 
    ## Director
personal['Director'] = 0
personal.loc[(personal['descargo']=='DIRECTOR I.E.') ,'Director'] =1 
    ## Tecnologos medicos
personal['Tecnologo medico'] = 0
personal.loc[(personal['descargo']=='PROFESIONAL DE TERAPIA FÍSICA PARA LOS PROGRAMAS DE INTERVENCIÓN TEMPRANA') | 
             (personal['descargo']=='TERAPISTA') | 
             (personal['descargo']=='FISIOTERAPEUTA'),'Tecnologo medico'] =1     
    ## Psicologos
personal['Psicologos'] = 0
personal.loc[(personal['descargo'].str.find('PSICOLOGO')!=-1) ,'Psicologos'] =1 
    ## Personal administrativo
personal['Administrativos'] = 0
personal.loc[(personal['descargo']=='TRABAJADOR DE SERVICIO') | 
             (personal['descargo']=='TRABAJADOR DE SERVICIO II') | 
             (personal['descargo']=='TRABAJADOR DE SERVICIO III') | 
             (personal['descargo']=='OFICINISTA') |                              
             (personal['descargo']=='TECNICO ADMINISTRATIVO'),'Administrativos'] =1

In [20]:
# Colapso el DataFrame a nivel de servicio educativo
trabajadores = personal.groupby('COD_MOD', as_index=False).agg('sum')
# Doy formato a la base
columnas = trabajadores.columns.tolist()
columnas_nexus = [columnas[0]] + columnas[-5:]
trabajadores = trabajadores[columnas_nexus]
trabajadores.sample(5)

,COD_MOD,Profesor,Director,Tecnologo medico,Psicologos,Administrativos
33,1329135,1,0,1,0,0
67,1702828,2,0,1,0,0
92,1753821,1,0,1,0,0
112,1793652,1,0,0,0,0
49,1527357,5,1,2,0,0


## Datos del padron web
Padron web descargado de la [pagina web](http://escale.minedu.gob.pe/uee/-/document_library_display/GMv7/view/958881) de escale con fecha 17/09/2022 y complemetado con información de las asignaciones temporales y unidades ejecutoras de la Unidad de Presupuesto

In [ ]:
# Importo el padron de la Unidad de Presupuesto 
df_padron = pd.read_stata(data/r'Padron GG1.dta')
df_padron = df_padron.loc[df_padron['niv_mod']=='E0']

In [26]:
# Identifico a los servicios PRITE
padron = pd.merge(left= ssee_prite, right=df_padron, left_on='COD_MOD', right_on='cod_mod', how='left', validate='1:m',indicator=True)
# Filtro las columnas relevantes
datos_padron = ['COD_MOD','cen_edu','d_gestion','d_ges_dep','ubigeo','d_dpto','d_prov', 'd_dist','d_region','codooii', 'd_dreugel','nlat_ie','nlong_ie',
                'vraem_upp_2022','tipie_upp_2022','bilin_upp_2022','rural_upp_2022','fron_upp_2022','region','pliego','codue', 'unidadejecutora', 'nombentidad']
padron_web = padron[datos_padron]

In [42]:
padron_web.sample(5)

,COD_MOD,cen_edu,d_gestion,d_ges_dep,ubigeo,d_dpto,d_prov,d_dist,d_region,codooii,...,vraem_upp_2022,tipie_upp_2022,bilin_upp_2022,rural_upp_2022,fron_upp_2022,region,pliego,codue,unidadejecutora,nombentidad
2,0780874,PRITE LUIS AQUILES GUERRA,P£blica de gesti¢n directa,Sector Educaci¢n,150112,LIMA,LIMA,INDEPENDENCIA,DRE LIMA METROPOLITANA,150103,...,No Vraem,Completo,No bilingue,Urbano,No frontera,LIMA METROPOLITANA,10. LIMA METROPOLITANA,102,002. USE 02 SAN MARTIN DE PORRAS,USE 02 SAN MARTIN DE PORRAS
56,1585827,PRITE UNA LUZ EN MI CAMINO,P£blica de gesti¢n directa,Sector Educaci¢n,220101,SAN MARTIN,MOYOBAMBA,MOYOBAMBA,DRE SAN MARTIN,220001,...,No Vraem,Completo,No bilingue,Urbano,No frontera,SAN MARTIN,459. GOBIERNO REGIONAL DEL DEPARTAMENTO DE SAN...,459300,300. EDUCACION SAN MARTIN,REGION SAN MARTIN-EDUCACION
11,1031749,PRITE ALDEA INFANTIL SAGRADA FAMILIA,P£blica de gesti¢n directa,Sector Educaci¢n,040103,AREQUIPA,AREQUIPA,CAYMA,DRE AREQUIPA,040001,...,No Vraem,Completo,No bilingue,Urbano,No frontera,AREQUIPA,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,443302,302. EDUCACION AREQUIPA NORTE,REGION AREQUIPA-EDUCACION AREQUIPA NORTE
101,1788660,PRITE MARIA JERUSALEN,P£blica de gesti¢n directa,Sector Educaci¢n,080604,CUSCO,CANCHIS,MARANGANI,DRE CUSCO,080006,...,No Vraem,Completo,No bilingue,Urbano,No frontera,CUSCO,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,446302,302. EDUCACION CANCHIS,REGION CUSCO-EDUCACION CANCHIS
44,1495373,PRITE MANUEL GONZALES PRADA,P£blica de gesti¢n directa,Sector Educaci¢n,150110,LIMA,LIMA,COMAS,DRE LIMA METROPOLITANA,150105,...,No Vraem,Completo,No bilingue,Urbano,No frontera,LIMA METROPOLITANA,10. LIMA METROPOLITANA,104,004. USE 04 COMAS,USE 04 COMAS


Consolidacion de la base de datos en el 2022

In [83]:
df = pd.merge(left=padron_web, right=df_mat_2022, on='COD_MOD')
df = pd.merge(left=df, right=trabajadores, on='COD_MOD')
df.sample(4)

,COD_MOD,cen_edu,d_gestion,d_ges_dep,ubigeo,d_dpto,d_prov,d_dist,d_region,codooii,...,Entre 1 año a 2 años,Entre 2 años a 3 años,Entre 4 años a 5 años,Matricula reportada,Año,Profesor,Director,Tecnologo medico,Psicologos,Administrativos
91,1768217,PRITE 50163 VIRGEN DE FATIMA,P£blica de gesti¢n directa,Sector Educaci¢n,080408,CUSCO,CALCA,YANATILE,DRE CUSCO,080004,...,5,0.0,0.0,12,2022,3,1,1,0,0
101,1788678,PRITE PITUMARCA,P£blica de gesti¢n directa,Sector Educaci¢n,080605,CUSCO,CANCHIS,PITUMARCA,DRE CUSCO,080006,...,9,0.0,0.0,47,2022,3,0,1,0,0
89,1765155,VILLA DEL SALVADOR,P£blica de gesti¢n directa,Sector Educaci¢n,021809,ANCASH,SANTA,NUEVO CHIMBOTE,DRE ANCASH,020018,...,28,0.0,0.0,42,2022,2,0,1,0,0
37,1386481,PRITE HOSPITAL REGIONAL,P£blica de gesti¢n directa,Sector Educaci¢n,080101,CUSCO,CUSCO,CUSCO,DRE CUSCO,080001,...,27,0.0,0.0,59,2022,5,1,2,0,0


In [77]:
df_mat_2022.sample(3)

,COD_MOD,Menor a 12 meses,Entre 12 meses a 1 año,Entre 1 año a 2 años,Entre 2 años a 3 años,Entre 4 años a 5 años,Matricula reportada,Año
0,0679092,3.0,5.0,5,0.0,0.0,13,2022
62,1627397,10.0,9.0,16,0.0,0.0,35,2022
86,1747112,3.0,7.0,11,1.0,0.0,22,2022


In [78]:
trabajadores.sample(3)

,COD_MOD,Profesor,Director,Tecnologo medico,Psicologos,Administrativos
2,0780874,8,0,1,0,2
20,1088566,6,0,1,0,1
13,1036417,4,1,1,1,0


In [79]:
padron_web.sample(3)

,COD_MOD,cen_edu,d_gestion,d_ges_dep,ubigeo,d_dpto,d_prov,d_dist,d_region,codooii,...,vraem_upp_2022,tipie_upp_2022,bilin_upp_2022,rural_upp_2022,fron_upp_2022,region,pliego,codue,unidadejecutora,nombentidad
96,1778950,PRITE MADRE DE DIOS,P£blica de gesti¢n directa,Sector Educaci¢n,170101,MADRE DE DIOS,TAMBOPATA,TAMBOPATA,DRE MADRE DE DIOS,170001,...,No Vraem,Completo,No bilingue,Urbano,No frontera,MADRE DE DIOS,454. GOBIERNO REGIONAL DEL DEPARTAMENTO DE MAD...,454300,300. EDUCACION MADRE DE DIOS,REGION MADRE DE DIOS-EDUCACION
72,1730720,PRITE,P£blica de gesti¢n directa,Sector Educaci¢n,110101,ICA,ICA,ICA,DRE ICA,110001,...,No Vraem,Completo,No bilingue,Urbano,No frontera,ICA,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,449300,300. EDUCACION ICA,REGION ICA-EDUCACION
100,1787894,PRITE SAGRADA FAMILIA,P£blica de gesti¢n directa,Sector Educaci¢n,080801,CUSCO,ESPINAR,ESPINAR,DRE CUSCO,080008,...,No Vraem,Completo,No bilingue,Urbano,No frontera,CUSCO,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,446310,310. EDUCACION ESPINAR,GOB.REG. DPTO. CUSCO- EDUCACION ESPINAR


In [253]:
# Reviso las observaciones
print('lista original',ssee_prite.shape[0], '\n', 'lista en nexus', trabajadores.shape[0])

lista original 117 
 lista en nexus 117


114

117

both          518
left_only       1
right_only      0
Name: _merge, dtype: int64

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_24640\3606240390.py:1: DtypeWarning: Columns (1,122) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(work/r'Results\\Base consolidada.csv')
